In [1]:
# Only when is Local
import findspark

findspark.init()
findspark.find()

'E:\\LibreriasPython\\spark-3.1.2-bin-hadoop2.7\\python\\pyspark'

In [10]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# **02 Poblado de Capas DataLake con Spark**
---------------------------------------

**Agenda**
- Poblado Capa Landing
- Poblado Capa Raw
- Poblado Capa Master o Silver
- Poblado Capa Business


## **Poblado Data Lake: Caso Data Coronavirus**
----------------------------------------

Lo han contratado como ingeniero de datos para realizar el poblado del datalake con información de coranavirus con el fin de que el negocio pueda tener la información disponible para sus modelos análiticos de datos.


Con este fin se le solicita poblar el datalake:
- Generar [ingesta de los Datos](https://docs.google.com/drawings/d/1oguRlE-4iIpcwtyPVBKvKUOSoAkm_HxHe4aYF5vNe-4/edit) (Poblado landing, raw y master)
- Realizar la lógica de negocio solicitada (Capa business)


In [ ]:
# Instalamos Spark
# !pip install pyspark py4j

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 45.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=68ff0f82469bd99e9489d85659845ec8e7fd5cf50383aa8ac494b7b602700536
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# Inicializamos Spark Session
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import shutil


# inicializamos datos
spark = SparkSession.builder.appName('spark_test').getOrCreate()

In [4]:
# Creamos zonas data lake
folders = ['landing', 'raw', 'master', 'business']
for folder in folders:
    #     path_temp = path + folder
    #     if not os.path.isdir(folder):
    os.mkdir(folder)


print(os.getcwd())
print(os.listdir())

E:\Repositorio\Pyspark\Sesiones\Sesion4
['.ipynb_checkpoints', '01 Arquitecturas Procesamiento Big Data.ipynb', '02 Poblado Capas Data Lake con Spark.ipynb', '03 Testeo Pipeline Spark.ipynb', 'business', 'Case.csv', 'Excel Procesamiento.xlsx', 'FLUJO INGESTA DE DATO.pdf', 'Flujo Procesamiento.pdf', 'Laboratorio.ipynb', 'landing', 'master', 'PatientInfo.csv', 'Policy.csv', 'raw', 'Region.csv', 'SearchTrend.csv', 'SeoulFloating.csv', 'test_demo', 'Time.csv', 'TimeAge.csv', 'TimeGender.csv', 'TimeProvince.csv', 'Weather.csv']


## 1. **Poblado Capa Landing**
-------------------------

- Cargue los Archivos `.csv` pertenecientes a Coronavirus Dataset
- Genere un pipeline de datos para cargar los archivos en la zona corresponfiente a landing



In [34]:
# Herramientas para mover información de un lugar a otro como sqoop o apache kafka
new_path = "E:\\Repositorio\\Pyspark\\Sesiones\\data\\"

In [ ]:
# Cargando data Google Colab
uploaded = files.upload()

In [37]:
os.chdir(new_path)

In [19]:
print(os.getcwd())

E:\Repositorio\Pyspark\Sesiones\Sesion4


In [6]:
for file in os.listdir():
  if '.csv' in file:
    shutil.move(file, f'./landing/{file}')

## **2. Poblado Capa Raw**
-----------------------

- **Genere los esquemas pertinentes** para cada uno de los archivos.
- **Renombre las columnas para cada archivo a fin de estandarizar los nombres** en caso lo considere
- Guarde cada tabla en la capa Raw usando el formato `avro`
- Nombre de tabla  `csv_file_name + table + raw` Ejemplo: case_table_raw
- Agregue un campo a manera de auditoria `AuditDate -> timestamp`



In [12]:
# lectura de archivo en landing

from pyspark.sql.types import StructField, StructType, StringType, TimestampType
from pyspark.sql import functions as F
from datetime import datetime


path = './landing/Case.csv'

options = {
    'header': True,
    'sep': ','
}
schema = StructType([
    StructField("case_id", StringType()),
    StructField("province", StringType()),
    StructField("city", StringType()),
    StructField("group", StringType()),
    StructField("infection_case", StringType()),
    StructField("confirmed", StringType()),
    StructField("latitude", StringType()),
    StructField("longitude", StringType()),
])


df = spark.read.options(**options).schema(schema).csv(path)

# renombrar columnas

In [13]:
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [16]:
# Columnas de auditoria

df = df.withColumn('auditDate', F.to_timestamp( F.lit(datetime.now())) )

In [17]:
df.show(truncate = False,vertical=True)

-RECORD 0-------------------------------------------------------
 case_id        | 1000001                                       
 province       | Seoul                                         
 city           | Yongsan-gu                                    
 group          | TRUE                                          
 infection_case | Itaewon Clubs                                 
 confirmed      | 139                                           
 latitude       | 37.538621                                     
 longitude      | 126.992652                                    
 auditDate      | 2022-02-13 17:17:20.150681                    
-RECORD 1-------------------------------------------------------
 case_id        | 1000002                                       
 province       | Seoul                                         
 city           | Gwanak-gu                                     
 group          | TRUE                                          
 infection_case | Richway

In [20]:
# guardar en zona raw
df.write.mode('overwrite').parquet('./raw/case_table_raw')

In [21]:
print(os.listdir('raw'))

['case_table_raw']


In [22]:
print(os.listdir('raw/case_table_raw'))

['.part-00000-d8476ea2-7a7c-4391-adb6-567ff75b813e-c000.snappy.parquet.crc', '._SUCCESS.crc', 'part-00000-d8476ea2-7a7c-4391-adb6-567ff75b813e-c000.snappy.parquet', '_SUCCESS']


In [23]:
class LoadRawData:
  def __init__(self, path: str):
    self.input_path = path
    pass
  
  def extract_df(self, schema, options):
    """
    Extract DataFrame
    """
    df = spark.read.options(**options).schema(schema).csv(self.input_path)
    self.extractDf = df
    pass

  def transform_df(self):
    """
    Transform Df
    """

    # transformando Df
    transformDF = (self.extractDf
                        .withColumn( 'auditDate', 
                                    F.to_timestamp( F.lit(datetime.now())) 
                                    )
                        )
    
    self.transformDf = transformDF
    pass

  
  def load_df(self, out_path):
    df =  self.transformDf
    df.write.mode('overwrite').parquet(out_path)
    pass
  pass


In [24]:
# case.csv



options = {
    'header': True,
    'sep': ','
}
schema = StructType([
    StructField("case_id", StringType()),
    StructField("province", StringType()),
    StructField("city", StringType()),
    StructField("group", StringType()),
    StructField("infection_case", StringType()),
    StructField("confirmed", StringType()),
    StructField("latitude", StringType()),
    StructField("longitude", StringType()),
])

# ETL
loadCase = LoadRawData('./landing/Case.csv')
loadCase.extract_df(schema, options)
loadCase.transform_df()
loadCase.load_df('./raw/case_table_raw')





In [25]:
# Region.csv



options = {
    'header': True,
    'sep': ','
}
schema = StructType([
    StructField('code', StringType()),
    StructField('province', StringType()),
    StructField('city', StringType()),
    StructField('latitude', StringType()),
    StructField('longitude', StringType()),
    StructField('elementary_school_count', StringType()),
    StructField('kindergarten_count', StringType()),
    StructField('university_count', StringType()),
    StructField('academy_ratio', StringType()),
    StructField('elderly_population_ratio', StringType()),
    StructField('elderly_alone_ratio', StringType()),
    StructField('nursing_home_count', StringType())
])
# ETL
loadCase = LoadRawData('./landing/Region.csv')
loadCase.extract_df(schema, options)
loadCase.transform_df()
loadCase.load_df('./raw/region_table_raw')

In [26]:
# TimeProvince.csv



options = {
    'header': True,
    'sep': ','
}
schema = StructType([
    StructField('date', StringType()),
    StructField('time', StringType()),
    StructField('province', StringType()),
    StructField('confirmed', StringType()),
    StructField('released', StringType()),
    StructField('deceased', StringType())
])


# ETL
loadCase = LoadRawData('./landing/TimeProvince.csv')
loadCase.extract_df(schema, options)
loadCase.transform_df()
loadCase.load_df('./raw/time_province_table_raw')





In [27]:
print(os.getcwd())
print(os.listdir('./raw/'))

E:\Repositorio\Pyspark\Sesiones\Sesion4
['case_table_raw', 'region_table_raw', 'time_province_table_raw']


## **3. Poblado Capa Master**
-----------------------------
Para cada tabla en la capa raw: 

- **Establesca su esquema** correspondiente (tipo de dato apropiado)
- **Genere columnas de auditoria** 
- Nombre de tabla  `csv_file_name + table + master` 
  
  **Ejemplo:**
  
    case_table_master

- Almanece los datos en formato `parquet`

In [28]:
# case_table_raw

from pyspark.sql.types import StructField, StructType, StringType, TimestampType, IntegerType, BooleanType, FloatType
from pyspark.sql import functions as F
from datetime import datetime



schema = StructType([
    StructField("case_id", StringType()),
    StructField("province", StringType()),
    StructField("city", StringType()),
    StructField("group", StringType()),
    StructField("infection_case", StringType()),
    StructField("confirmed", StringType()),
    StructField("latitude", StringType()),
    StructField("longitude", StringType()),
])

# leer schema anterio - > cast a las columnas segun corresponda

casosTableDf = spark.read.schema(schema).parquet('./raw/case_table_raw')


# Nuevo schema con la definición correcta de los datos
dicx_case_cast ={
    'case_id': IntegerType(),
    'province': StringType(),
    'city': StringType(),
    'group': BooleanType(),
    'infection_case': StringType(),
    'confirmed': IntegerType(),
    'latitude': FloatType(),
    'longitude': FloatType()
} 


transformDf = casosTableDf
for key, value in dicx_case_cast.items():
  transformDf = transformDf.withColumn(key, F.col(key).cast(value))

# logica -> 
# auditoria
transformDf.withColumn('auditDate', 
                                    F.to_timestamp( F.lit(datetime.now())) 
                                    )


# guardamos en ruta
transformDf.write.mode('overwrite').parquet('./master/case_table_master')

In [29]:
casosTableDf.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [30]:
os.listdir('master/case_table_master')

['.part-00000-7528bffc-a023-48bb-93fa-c1c6c6a7b81f-c000.snappy.parquet.crc',
 '._SUCCESS.crc',
 'part-00000-7528bffc-a023-48bb-93fa-c1c6c6a7b81f-c000.snappy.parquet',
 '_SUCCESS']

In [31]:

# region_table_raw

from pyspark.sql.types import StructField, StructType, StringType, TimestampType, IntegerType, BooleanType, FloatType
from pyspark.sql import functions as F
from datetime import datetime



schema = StructType([
    StructField('code', StringType()),
    StructField('province', StringType()),
    StructField('city', StringType()),
    StructField('latitude', StringType()),
    StructField('longitude', StringType()),
    StructField('elementary_school_count', StringType()),
    StructField('kindergarten_count', StringType()),
    StructField('university_count', StringType()),
    StructField('academy_ratio', StringType()),
    StructField('elderly_population_ratio', StringType()),
    StructField('elderly_alone_ratio', StringType()),
    StructField('nursing_home_count', StringType())
])

dicx_region_cast ={
    "code": IntegerType() ,
    "province": StringType(),
    "city": StringType(),
    "latitude": FloatType(),
    "longitude": FloatType(),
    "elementary_school_count": IntegerType(),
    "kindergarten_count": IntegerType(),
    "university_count": IntegerType(),
    "academy_ratio": FloatType(),
    "elderly_population_ratio": FloatType(),
    "elderly_alone_ratio": FloatType(),
    "nursing_home_count": IntegerType()
} 


# leer schema anterio - > cast a las columnas segun corresponda

regionTableDf = spark.read.schema(schema).parquet('./raw/region_table_raw')

transformDf = regionTableDf
for key, value in dicx_region_cast.items():
  transformDf = transformDf.withColumn(key, F.col(key).cast(value))

# logica -> 
# auditoria
transformDf.withColumn('auditDate', 
                                    F.to_timestamp( F.lit(datetime.now())) 
                                    )


# guardamos en ruta
transformDf.write.mode('overwrite').parquet('./master/region_table_master')

## **4. Poblado Capa Business**
-----------------------------------

COVID-19 ha infectado a más de 10,000 personas en Corea del Sur.
KCDC (Centros de Corea para el Control y la Prevención de Enfermedades) anuncia la información de COVID-19 de forma rápida y transparente.

Se le solicita generar un tablon siguiendo ciertas lógicas de negocio

- El cliente le ha brindado el siguiente [esquema de relación de los datos](https://docs.google.com/drawings/d/1ryklgBvchXfmPgh4YkLq8-cn_OuyTohanugkqv5KzDs/edit)
- Se le brinda la [lógica de la solución](https://docs.google.com/spreadsheets/d/1KAWJn1rzQPdmC-A6m9wy_Vx9Bkon81676NxNyhFIMk0/edit#gid=0)
- Genere el pipeline correspondiente a fin de obtener la tabla final solicitada
- Almacene en formato `parquet`



Se le brinda la siguiente imagen que relaciona los datasets a manera de contextualizar la solicitud

<img src='https://miro.medium.com/max/1400/0*RVqtYcbfYPdKuU2L.PNG' width=700>


In [ ]:
# TODO

In [4]:
from pyspark.sql.types import StructField, StructType, StringType, TimestampType, IntegerType, BooleanType, FloatType
from pyspark.sql import functions as F
from datetime import datetime

region_schema = StructType([
    StructField('code', IntegerType()),
    StructField('province', StringType()),
    StructField('city', StringType()),
    StructField('latitude', FloatType()),
    StructField('longitude', FloatType()),
    StructField('elementary_school_count', IntegerType()),
    StructField('kindergarten_count', IntegerType()),
    StructField('university_count', IntegerType()),
    StructField('academy_ratio', FloatType()),
    StructField('elderly_population_ratio', FloatType()),
    StructField('elderly_alone_ratio', FloatType()),
    StructField('nursing_home_count', IntegerType())
])



case_schema = StructType([
    StructField("case_id", IntegerType()),
    StructField("province", StringType()),
    StructField("city", StringType()),
    StructField("group", BooleanType()),
    StructField("infection_case", StringType()),
    StructField("confirmed", IntegerType()),
    StructField("latitude", FloatType()),
    StructField("longitude", FloatType()),
])

def readParquet(path,schema):
    return spark.read.schema(schema).parquet(path)

# Lectura de inputs
case_colums = ["case_id","province","city","group","infection_case","confirmed","latitude","longitude"]
region_columns = ["province","city","elementary_school_count","kindergarten_count","university_count","academy_ratio","elderly_population_ratio","elderly_alone_ratio","nursing_home_count"]

caseDF = readParquet("master/case_table_master",case_schema).select(*case_colums)
regionDF = readParquet("master/region_table_master",region_schema).select(*region_columns)

In [5]:
# Transformaciones
joinDF = caseDF.join(regionDF,['province','city'],'inner')
joinDF.printSchema()

root
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- elementary_school_count: integer (nullable = true)
 |-- kindergarten_count: integer (nullable = true)
 |-- university_count: integer (nullable = true)
 |-- academy_ratio: float (nullable = true)
 |-- elderly_population_ratio: float (nullable = true)
 |-- elderly_alone_ratio: float (nullable = true)
 |-- nursing_home_count: integer (nullable = true)



In [9]:
# Filtrado parte 1
cond = (joinDF.latitude.isNotNull() | joinDF.longitude.isNotNull())
filterDF = joinDF.filter(cond)
filterDF.show(10,False)

+--------+----------+-------+-----+-------------------------------+---------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|province|city      |case_id|group|infection_case                 |confirmed|latitude |longitude |elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+--------+----------+-------+-----+-------------------------------+---------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|Seoul   |Gangnam-gu|1000029|true |Samsung Fire & Marine Insurance|4        |37.49828 |127.030136|33                     |38                |0               |4.18         |13.17                   |4.3                |3088              |
|Seoul   |Gangnam-gu|1000025|true |Gangnam Dongin Ch

In [11]:
print(joinDF.count())
print(filterDF.count())

68
63


In [12]:
# Filtrado parte 2
trasformDF = filterDF.withColumn('group',F.when(F.col('group')== F.lit("true"),F.lit('Infectado')).otherwise(F.lit('NoInfectado')))
trasformDF.show(10,False)

+--------+----------+-------+---------+-------------------------------+---------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|province|city      |case_id|group    |infection_case                 |confirmed|latitude |longitude |elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+--------+----------+-------+---------+-------------------------------+---------+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|Seoul   |Gangnam-gu|1000029|Infectado|Samsung Fire & Marine Insurance|4        |37.49828 |127.030136|33                     |38                |0               |4.18         |13.17                   |4.3                |3088              |
|Seoul   |Gangnam-gu|1000025|Infecta

In [19]:
# Auditoria
trasformDF = trasformDF.withColumn('AuditDate', 
                                    F.to_timestamp( F.lit(datetime.now())) 
                                    )
# Selección
columns_table = ["case_id","city","group","infection_case","confirmed","latitude","longitude",
                 "province","elementary_school_count","kindergarten_count","university_count",
                 "academy_ratio","elderly_population_ratio","elderly_alone_ratio","nursing_home_count","AuditDate"]
selectionDF = trasformDF.select(*columns_table) 
selectionDF.show(10,False)

+-------+----------+---------+-------------------------------+---------+---------+----------+--------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------------+
|case_id|city      |group    |infection_case                 |confirmed|latitude |longitude |province|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|AuditDate                 |
+-------+----------+---------+-------------------------------+---------+---------+----------+--------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------------+
|1000029|Gangnam-gu|Infectado|Samsung Fire & Marine Insurance|4        |37.49828 |127.030136|Seoul   |33                     |38                |0               |4.18         |13.17               

In [20]:
# Cargar en business zone
selectionDF.write.mode('overwrite').parquet('business/coronavirusTable')

## Relacionado

- Descargar la data en el siguiente [link](https://www.kaggle.com/kimjihoo/coronavirusdataset) 


